# Initiation aux système de recommandation

à partir du tuto :

https://www.datacamp.com/community/tutorials/recommender-systems-python

## 1. Recommandation Simple

Comme décrit dans la section précédente, les recommandation simples sont des systèmes de base qui recommandent les meilleurs éléments en fonction d'une certaine métrique ou d'un certain score. Dans cette section, vous allez créer un clone simplifié des 250 meilleurs films IMDB à l'aide de métadonnées collectées à partir d'IMDB. 

In [52]:
# Import Pandas
import pandas as pd
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from scipy import sparse
import time
import warnings
warnings.filterwarnings('ignore')


# Load Movies Metadata
metadata = pd.read_csv('./Datas/movies_metadata.csv', low_memory=False)

# Print the first three rows
metadata.head(2)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


#### Metric for film evaluation

WeightedRating (WR) :
\begin{equation}
    \displaystyle WR = \Big( \frac{v}{v+m}.R \Big) + \Big( \frac{v}{v+m}.C \Big) 
\end{equation}

where :
* $v$ is the number of votes for the movie;
* $m$ is the minimum votes required to be listed in the chart;
* $R$ is the average rating of the movie;
* $C$ is the mean vote across the whole report.


In [2]:
# Calculate mean of vote average column
C = metadata['vote_average'].mean()
print(C)

5.618207215134185


La note moyenne d'un film sur IMDB est d'environ 5,6 sur une échelle de 10. 

In [3]:
# Calculate the minimum number of votes required to be in the chart, m
m = metadata['vote_count'].quantile(0.90)
print(m)

160.0


Utiliser la méthode .copy() pour vous assurer que le nouveau DataFrame créé q_movies est indépendant des métadonnées du DataFrame d'origine.

In [4]:
# Filter out all qualified movies into a new DataFrame
q_movies = metadata.copy().loc[metadata['vote_count'] >= m]
print('Pour les film séléctionnés, {} est la dimension du nouveau DataFrame'.format(q_movies.shape))
print('Alors que la dimentsion du DataFrame complet est : ', metadata.shape)

Pour les film séléctionnés, (4555, 24) est la dimension du nouveau DataFrame
Alors que la dimentsion du DataFrame complet est :  (45466, 24)


Maintenant définisons la fonctionnalité weighted rating :

In [5]:
# Function that computes the weighted rating of each movie
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v / (v + m) * R) + (m / (m + v) * C)

Ajouter une nouvelle colone :

In [6]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

Et enfin, il faut trier le DataFrame par ordre décroissant en fonction du score et afficher le titre, le nombre de votes, la moyenne des votes et la note pondérée (score) des 20 meilleurs films. 

In [7]:
#Sort movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending=False)

#Print the top 15 movies
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(20)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
292,Pulp Fiction,8670.0,8.3,8.251406
522,Schindler's List,4436.0,8.3,8.206639
23673,Whiplash,4376.0,8.3,8.205404
5481,Spirited Away,3968.0,8.3,8.196055
2211,Life Is Beautiful,3643.0,8.3,8.187171


## 2. Recommandations basées sur le contenu

### Recommendations basé sur la description de parcelle 

Dans cette section, nous allons créer un système qui recommande des films similaires à un film particulier. Pour ce faire, nous calculerons des paire de similitude par cosinus_cores pour tous les films en fonction de leurs descriptions d'intrigue et recommanderons des films en fonction de ce seuil de score de similitude. 

In [8]:
#Print plot overviews of the first 5 movies.
metadata['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

Le problème à résoudre est un problème de traitement du langage naturel. Par conséquent, vous devez extraire une sorte de caractéristiques des données texte ci-dessus avant de pouvoir calculer la similitude et / ou la dissemblance entre elles. Pour faire simple, il n'est pas possible de calculer la similitude entre deux aperçus sous leur forme brute. Pour ce faire, vous devez calculer les vecteurs de mots de chaque vue d'ensemble ou document.

In [9]:
#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
metadata['overview'] = metadata['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(metadata['overview'])

In [10]:
#Output the shape of tfidf_matrix
print("La dimension de la matrice correspondant à la section overview 'according to the corpus' est :", tfidf_matrix.shape)

La dimension de la matrice correspondant à la section overview 'according to the corpus' est : (45466, 75827)


#### Taille en mémoire

In [11]:
tfidf_matrix.dtype

dtype('float64')

Un float64 prend 8 octets en mémoire

In [13]:
shape = tfidf_matrix.shape
shape[0]*shape[1]*8

27580403056

Cette matrice devrait prendre 27 Go en mémoire

In [14]:
tfidf_matrix

<45466x75827 sparse matrix of type '<class 'numpy.float64'>'
	with 1210882 stored elements in Compressed Sparse Row format>

Cette matrice étant une matrice sparce on ne stoque que 1210882 élément en mémoire 

Mais elle ne prend que (1210882*8) = 9687056 octets soit 9 Mo en étant stocquée comme une matrice sparse

In [16]:
#Array mapping from feature integer indices to feature name.
print('Exemple de mots du corpus :', tfidf.get_feature_names()[5000:5010])

Exemple de mots du corpus : ['avails', 'avaks', 'avalanche', 'avalanches', 'avallone', 'avalon', 'avant', 'avanthika', 'avanti', 'avaracious']


À partir de la sortie ci-dessus, on observe 75 827 mots de vocabulaires différents dans votre ensemble de données de 45 000 films.

On va maintenant calculer un score de similarité. Il existe plusieurs métriques de similitude que l'on peux utiliser pour cela, telles que les scores de similitude de Manhattan, euclidien, Pearson et cosinus. Encore une fois, il n'y a pas de bonne réponse pour savoir quel score est le meilleur. Différents scores fonctionnent bien dans différents scénarios, et c'est souvent une bonne idée d'expérimenter différentes mesures et d'observer les résultats. 

#### Exemple : Cosinus similarity

\begin{equation}
    \displaystyle \cos(x, y) = \frac{x.y^T}{||x||.||y||} = \frac{\sum_{i=1}^n x_i.y_i^{T}}{\sqrt{\sum_{i=1}^n (x_i)^2}\sqrt{\sum_{i=1}^n (y_i)^2}}
\end{equation}

#### 2.1.1. Matrice de cosine similarity complete

In [17]:
## Import linear_kernel
#from sklearn.metrics.pairwise import linear_kernel
#
## Compute the cosine similarity matrix
#cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix, dense_output=False)
#
#print('Dimensions of cosin similarity matrix :', cosine_sim.shape)
print("IMPOSSIBLE de calculer la matrice de cosinus similarity de cette manière -> EXPLOSE EN MEMOIRE")

IMPOSSIBLE de calculer la matrice de cosinus similarity de cette manière -> EXPLOSE EN MEMOIRE


In [18]:
en_ligne = False
dense_output_choice = False# True or False
'''
Ne pas utiliser dense_output = True
 -> EXPLOSE EN MEMOIRE
'''
if en_ligne==True:
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix, dense_output=dense_output_choice)


Complexité en mémoire O(n^2)

En l'executant sur google collab je trouve que la matrice de 45466*45466 élément (soit 2067157156 éléments) devrait prendre en mémoire 16537257248 octets (16,5 Go)

En étant stockée comme une matrice sparce seulement 546860044 éléments sont stocké, soit 546860044*8 = 4374880352 octets 
(4,4 Go) en plus de tout le reste...

**-> Sur notre machine ça ne passe pas en mémoire**

In [19]:
if en_ligne==True:
    print(cosine_sim[1,:5])

In [20]:
if en_ligne==True:
    cosine_sim[1,:].toarray()

Mécanisme pour identifier l'index d'un film dans votre metadataDataFrame, compte tenu de son titre. 

In [21]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()


In [22]:
tfidf_matrix[1,:]

<1x75827 sparse matrix of type '<class 'numpy.float64'>'
	with 33 stored elements in Compressed Sparse Row format>

In [23]:
indices[:10]

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
Heat                           5
Sabrina                        6
Tom and Huck                   7
Sudden Death                   8
GoldenEye                      9
dtype: int64

#### FONCTION DE RECOMMANDATION

Voici les étapes que vous suivrez:

* Obtenir l'index du film en fonction de son titre.

* Obtenir la liste des scores de similarité cosinus pour ce film particulier avec tous les films. Convertissez-le en une liste de tuples où le premier élément est sa position et le second est le score de similarité.

* Trier la liste susmentionnée de tuples en fonction des scores de similitude; c'est-à-dire le deuxième élément.

* Obtenir les 10 principaux éléments de cette liste. Ignorez le premier élément car il fait référence à soi (le film le plus similaire à un film particulier est le film lui-même).

* Renvoyer les titres correspondant aux indices des éléments supérieurs.


In [24]:
if en_ligne==True:
    # Function that takes in movie title as input and outputs most similar movies
    def get_recommendations(title, cosine_sim=cosine_sim):
        # Get the index of the movie that matches the title
        idx = indices[title]

        # Get the pairwsie similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[:,idx].toarray()))

        # Sort the movies based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Get the scores of the 10 most similar movies
        sim_scores = sim_scores[1:11]

        # Get the movie indices
        movie_indices = [i[0] for i in sim_scores]

        # Return the top 10 most similar movies
        return metadata['title'].iloc[movie_indices]

In [25]:
if en_ligne==True:
    get_recommendations('The Dark Knight Rises')

In [26]:
if en_ligne==True:
    get_recommendations('The Godfather')

#### 2.1.2. Calcul de cosine similarity pour un film donné (online)

In [27]:
# test cosin_sim sur 1 film
title = 'The Dark Knight Rises'
import time
t0 = time.time()
cosine_sim_1titre = cosine_similarity(tfidf_matrix[indices[title],:], tfidf_matrix, dense_output=dense_output_choice)
tps = time.time() - t0
print('temps execution', tps, 'secondes')

temps execution 0.0650491714477539 secondes


In [37]:
# Function that takes in movie title as input and outputs most similar movies
def get_reco_calcul_live(title, X=tfidf_matrix, nb_films=10):
    # Get the index of the movie that matches the title
    idx = indices[title]
    
    if X.shape == tfidf_matrix.shape:
        ## Compute the cosine similarity vector according to the current Film
        cosine_sim_1titre = cosine_similarity(X[idx,:], X)
    else:
        cosine_sim_1titre = cosine_similarity(X[idx,:].reshape(1, -1), X)

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim_1titre[0]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the n most similar movies
    sim_scores = sim_scores[1:nb_films+1]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return metadata.iloc[movie_indices]

In [38]:
Film = 'The Dark Knight Rises'
t0 = time.time()
list_reco = get_reco_calcul_live(Film)
tps = time.time() - t0
print('temps execution', tps, 'secondes')
print('Liste de recommandation pour le Film {} : {}'.format(Film, list_reco['title']))

temps execution 0.10975980758666992 secondes
Liste de recommandation pour le Film The Dark Knight Rises : 12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
15511                           Batman: Under the Red Hood
585                                                 Batman
21194    Batman Unmasked: The Psychology of the Dark Kn...
9230                    Batman Beyond: Return of the Joker
18035                                     Batman: Year One
19792              Batman: The Dark Knight Returns, Part 1
3095                          Batman: Mask of the Phantasm
Name: title, dtype: object


In [39]:
Film = 'The Godfather'
t0 = time.time()
list_reco = get_reco_calcul_live(Film)
tps = time.time() - t0
print('temps execution', tps, 'secondes')
print('Liste de recommandation pour le Film {} : {}'.format(Film, list_reco['title']))

temps execution 0.10727500915527344 secondes
Liste de recommandation pour le Film The Godfather : 1178               The Godfather: Part II
44030    The Godfather Trilogy: 1972-1990
1914              The Godfather: Part III
23126                          Blood Ties
11297                    Household Saints
34717                   Start Liquidation
10821                            Election
38030            A Mother Should Be Loved
17729                   Short Sharp Shock
26293                  Beck 28 - Familjen
Name: title, dtype: object


### Réduction de la dimension du corpus (using PCA / SVD)

In [31]:
#pca = PCA(n_components=0.99, svd_solver='full')
#pca.fit(tfidf_matrix)
'''Impossible de faire une PCA avec une matrice sparse'''


def SVD_scaling(n_comp_svd):
    if n_comp_svd>3000:
        n_comp_svd =3000
    from sklearn.decomposition import TruncatedSVD
    svd = TruncatedSVD(n_components=n_comp_svd, random_state=42)
    svd.fit(tfidf_matrix)
    print('Cumulative explained variance ratio :', svd.explained_variance_ratio_.sum())
    return svd


n_comp_svd = 2000
svd = SVD_scaling(n_comp_svd)

Cumulative explained variance ratio : 0.44246613343135216


Temps d'execution assez long mais effectué off-line donc ce n'est pas grave

In [32]:
X_trunc = svd.transform(tfidf_matrix)

In [33]:
Film = 'The Dark Knight Rises'
t0 = time.time()
list_reco = get_reco_calcul_live(Film, X_trunc)
tps = time.time() - t0
print('temps execution', tps, 'secondes')
print('Liste de recommandation pour le Film {} : {}'.format(Film, list_reco['title']))

temps execution 0.774378776550293 secondes
Liste de recommandation pour le Film The Dark Knight Rises : 150                                         Batman Forever
12481                                      The Dark Knight
585                                                 Batman
15511                           Batman: Under the Red Hood
1328                                        Batman Returns
21194    Batman Unmasked: The Psychology of the Dark Kn...
25267                                    Batman vs Dracula
35983                                    Batman: Bad Blood
19792              Batman: The Dark Knight Returns, Part 1
18035                                     Batman: Year One
Name: title, dtype: object


Les recommandations semblent cohérentes, donc les performances de recommandation sont correctes. En revanche le temps de calcul pour la fonction de recommandation est multiplié par 8 (alors qu'on souhaitait dimininuer). En effet la matrice du corpus vectorisé est plus petite (seulement 2000 features) mais il s'agit d'une matrice dense, ainsi lors du calcul de similarité il y a plus de calcul à effectuer qu'en présence d'une matrice sparse.

#### Avec peu de composantes pour le SVD

In [34]:
n_comp_svd = 100
svd = SVD_scaling(n_comp_svd)
X_trunc = svd.transform(tfidf_matrix)

Cumulative explained variance ratio : 0.07727909798893641


In [43]:
Film = 'The Dark Knight Rises'
t0 = time.time()
list_reco = get_reco_calcul_live(Film, X_trunc)
tps = time.time() - t0
print('temps execution', tps, 'secondes')
print('Liste de recommandation pour le Film {} : {}'.format(Film, list_reco['title']))

temps execution 0.11269736289978027 secondes
Liste de recommandation pour le Film The Dark Knight Rises : 35983                         Batman: Bad Blood
23897              Teenage Mutant Ninja Turtles
40437                                   City 40
23717                      Deliver Us from Evil
18035                          Batman: Year One
32601                       Cyber City Oedo 808
40095    Hieronymus Bosch: Touched by the Devil
20993                        The Lords of Salem
37775                Three Cheers for the Irish
39809                          TransFatty Lives
Name: title, dtype: object


Pas de gain en temps => la plupart / tous des vecteurs sont composés de moins de 100 mots

De plus les recommandation sont maintenant dégradés => Trop de perte d'information avec seulement 100 composantes pour le SVD (explained variance ratio : 0.07727909798893641)

## Améliorations

### 1. Introduce a popularity filter: 
#### Simple way 
This recommender would take the 30 most similar movies, calculate the weighted ratings (using the IMDB formula from above), sort movies based on this rating, and return the top 10 movies.


In [53]:
def get_popular_correlate_reco(Film):
    list_reco = get_reco_calcul_live(Film, nb_films = 30)
    list_reco['score'] = list_reco.apply(weighted_rating, axis=1)
    list_reco = list_reco.sort_values('score', ascending=False)
    return list_reco.head(10)

Film = 'The Dark Knight Rises'
list_reco = get_popular_correlate_reco(Film)
print('Liste de recommandation pour le Film {} ordonnés par populatité : '.format(Film))
list_reco[['title', 'score']]

Liste de recommandation pour le Film The Dark Knight Rises ordonnés par populatité : 


,title,score
12481,The Dark Knight,8.265477
10122,Batman Begins,7.460750
20232,"Batman: The Dark Knight Returns, Part 2",7.276985
19792,"Batman: The Dark Knight Returns, Part 1",7.115637
15511,Batman: Under the Red Hood,7.087743
41976,The Lego Batman Movie,7.045017
585,Batman,6.904084
3095,Batman: Mask of the Phantasm,6.645802
9230,Batman Beyond: Return of the Joker,6.534978
18035,Batman: Year One,6.528706


#### More efficient : somme pondérée du score de similarité et du score de popularité

In [81]:
def get_mix_popular_correlate_reco(title, X=tfidf_matrix, nb_films=30, alpha=0.5):
    # Get the index of the movie that matches the title
    idx = indices[title]
    
    ## Compute the cosine similarity vector according to the current Film
    cosine_sim_1titre = cosine_similarity(X[idx,:], X)
    
    # Get the WR score 
    WR = metadata.apply(weighted_rating, axis=1) / 10 
    
    # Combinaison linéaire de sim_score et popularity score
    combin_scores = alpha * cosine_sim_1titre + (1 - alpha) * WR.values
    
    # Get the pairwsie similarity scores of all movies with that movie
    sim_WR_scores = list(enumerate(combin_scores[0]))
    
    # Sort the movies based on the linear combinaison of similarity scores
    # and popularity score
    sim_WR_scores = sorted(sim_WR_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the n most similar movies
    sim_WR_scores = sim_WR_scores[1:nb_films+1]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_WR_scores]

    # Return the top 10 most similar movies
    return metadata.iloc[movie_indices]

## poids de la similarité par rapport à la popularité
## alpha * similarity score + (1-alpha) * popularity score
alpha = 0.6 

Film = 'Forrest Gump' ## Fight Club ## Forrest Gump ## The Dark Knight Rises
list_reco = get_mix_popular_correlate_reco(Film, alpha=alpha)
print("Liste de recommandation pour le Film '{}' ordonnés par populatité :".format(Film))
list_reco[['title']].head(10)

Liste de recommandation pour le Film 'Forrest Gump' ordonnés par populatité :


,title
32144,Room
18465,The Intouchables
5851,Catch Me If You Can
20779,Wolf Children
7834,The Notebook
522,Schindler's List
22110,Frozen
10309,Dilwale Dulhania Le Jayenge
1624,Good Will Hunting
314,The Shawshank Redemption


 ### 2. get_recommandation with title suggestion 

In [83]:
#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf2 = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
metadata['title'] = metadata['title'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_title = tfidf2.fit_transform(metadata['title'])

tfidf_title

<45466x22834 sparse matrix of type '<class 'numpy.float64'>'
	with 92758 stored elements in Compressed Sparse Row format>

In [ ]:
alpha = 0.6 
try:
    title_film = input('enter the title of the movie: ')
    list_reco = get_mix_popular_correlate_reco(title_film, alpha=alpha)
    print("List recommandation for Film '{}' :".format(Film))
    list_reco[['title']].head(10)
except:
    print('Do you mean :')
    title = tfidf2.transform([title_film])
    list_film_potentiels = get_reco_calcul_live(title, X=tfidf_title, nb_films=10)
    print(list_film_potentiels[['title']])
    
    
    